In [1]:
import numpy as np
import tensorflow as tf

from tqdm import tqdm

from batcher import Batcher

In [2]:
batch_size = 64

In [3]:
X_train = np.load('./X_train.npy')
y_train = np.load('./y_train.npy')

X_val = np.load('./X_val.npy')
y_val = np.load('./y_val.npy')

n_examples = y_train.shape[0]
n_time_steps = y_train.shape[1]
n_ftrs = X_train.shape[2]
n_bids = y_train.shape[2]

In [4]:
n_examples, n_time_steps, n_ftrs, n_bids

(400000, 4, 179, 40)

In [5]:
lstm_size = 64
n_layers = 3

keep_prob = tf.placeholder(tf.float32, name='keep_prob')

cells = []
for _ in range(n_layers):
    cell = tf.contrib.rnn.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(lstm_size),
        output_keep_prob=keep_prob
    )
    cells.append(cell)
    
lstm_cell = tf.contrib.rnn.MultiRNNCell(cells)

seq_in = tf.placeholder(tf.float32, [None, None, n_ftrs], 'seq_in')
seq_out = tf.placeholder(tf.float32, [None, None, n_bids], 'seq_out')

softmax_w = tf.get_variable('softmax_w', shape=[lstm_cell.output_size, n_bids], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))

In [6]:
#out_rnn, _ = tf.nn.dynamic_rnn(lstm_cell, seq_in, initial_state=lstm_cell.zero_state(batch_size, dtype=tf.float32))
out_rnn, _ = tf.nn.dynamic_rnn(lstm_cell, seq_in, dtype=tf.float32)

In [7]:
out_bid_logit = tf.matmul(tf.reshape(out_rnn, [-1, lstm_size]), softmax_w, name='out_bid_logit')

In [8]:
out_bid_target = tf.reshape(seq_out, [-1, n_bids])

In [9]:
cost = tf.losses.softmax_cross_entropy(out_bid_target, out_bid_logit)

In [10]:
#train_step = tf.train.MomentumOptimizer(learning_rate=0.001, momentum=0.9).minimize(cost)
train_step = tf.train.AdamOptimizer(0.001).minimize(cost)

In [11]:
batch = Batcher(n_examples, batch_size)
cost_batch = Batcher(n_examples, 10000)

In [12]:
sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())

In [13]:
for i in range(5000):
    x_batch, y_batch = batch.next_batch([X_train, y_train])
    if i % 1000 == 0:
        x_cost, y_cost = cost_batch.next_batch([X_train, y_train])
        c_train = sess.run(cost, feed_dict={seq_in: x_cost, seq_out: y_cost, keep_prob: 1.0})
        c_valid = sess.run(cost, feed_dict={seq_in: X_val, seq_out: y_val, keep_prob: 1.0})
        print('{}. c_train={} c_valid={}'.format(i, c_train, c_valid))
    sess.run(train_step, feed_dict={seq_in: x_batch, seq_out: y_batch, keep_prob: 0.6})

0. c_train=3.6880698204040527 c_valid=3.688000440597534
1000. c_train=0.7143563628196716 c_valid=0.7177790999412537
2000. c_train=0.6318325400352478 c_valid=0.6299329400062561
3000. c_train=0.5171809792518616 c_valid=0.5199329853057861
4000. c_train=0.45557910203933716 c_valid=0.45627012848854065


In [14]:
saver = tf.train.Saver()
saver.save(sess, 'model/test')

'model/test'